In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import math
import random
from __future__ import division, print_function

plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
D = 1.0
xi = 2.

def update(x, dt, f=None):
    if f is None:
        return x + 2*(np.random.random_sample(x.shape)-0.5)*math.sqrt(dt*D)
    else:
        return x + 2*(np.random.random_sample(x.shape)-0.5)*math.sqrt(dt*D) + f(x)*dt/(xi)


def srk(x, dt, f):
    f1 = f(x)
    G0 = (np.random.normal(0, 1, x.shape))*math.sqrt(2*D*dt)
    x1 = x + D/T*f(x)*dt + G0
    f2 = f(x1)
    return x + D/(2*T)*(f1+f2)*dt + G0

def f_hat(x):
    rsq = np.sum(x**2, axis=1).reshape((-1,1))
    return 3*(2*x - 1.2*rsq*x)

def f_zero(x):
    return 0

def f_hat_rotation(x):
    v = 5. * x[:,::-1]
    v[:,1] *= -1
    return f_hat(x) + v

def create_f_hat(alpha, gamma):
    def f(x):
        rsq = np.sum(x**2, axis=1).reshape((-1,1))
        return alpha*x - gamma*rsq*x
    return f

def create_f_hat_rotation(alpha, gamma, k):
    f = create_f_hat(alpha, gamma)
    def g(x):
        v = k * x[:,::-1]
        v[:,1] *= -1
        return f(x) + v
    return g


In [ ]:
x0 = np.random.random_sample((23, 2))
x0 = np.zeros((32, 2))

DT = 0.05
T = 1.

msd_data = []

for i in range(10):
    x = x0.copy()
    data = [x]
    for t in range(5000):
        x = srk(x, DT, f_hat_rotation)
        data.append(x)
    data = np.array(data)
    msd_data.append((data**2).sum(axis=-1).mean(axis=-1))
msd_data = np.array(msd_data)

In [ ]:
plt.hist2d(data[...,0].reshape((-1,)), data[...,1].reshape((-1,)), bins=32);

In [ ]:
[plt.plot(data[:,i,0], data[:,i,1]) for i in range(2)];

In [ ]:
tt = DT*np.arange(len(data))
m = np.mean(msd_data, axis=0)
s = np.std(msd_data, axis=0)
plt.plot(tt, m)
plt.plot(tt, m-s)
plt.plot(tt, m+s)
plt.plot(tt, tt * D, 'k')
plt.plot(tt, 2 * tt * D * (1+2*math.sqrt(1./(msd_data.shape[0]*data.shape[1]))), 'k--')
plt.plot(tt, 2 * tt * D * (1-2*math.sqrt(1./(msd_data.shape[0]*data.shape[1]))), 'k--')

In [ ]:
x0 = np.array( (1, 0) )

x = x0.copy()

DT=0.2

K = 1.

def trap(x):
    return - K * x

# hat potential = 0.3*x**4 - x**2
# hat force = - (1.2 x**3 - 2 x) = 2 x - 1.2 x**3

def hat(x):
    rsq = sum(x**2)
    return 2*x - 1.2*rsq*x

data = []
for t in range(1000):
    x = update(x, DT, hat)
    data.append(x)

In [ ]:
plt.plot(data[:,1,:])

In [ ]:
np.sum?

In [ ]:
msd_data.shape

In [ ]:
data.shape

In [ ]:
D_tracer = 0.0001
D_bath = 1.
T = 1
def srk_w_tracer(x, X, dt, bond_k, f):
    f1 = f(x)
    bond_f1 = bond_k * (X.reshape((1,-1)) - x) / x.shape[0]
    f1 += bond_f1
    F1 = -bond_f1.sum(axis=0)
    g0 = (np.random.normal(0, 1, x.shape))*math.sqrt(2*D_bath*dt)
    G0 = (np.random.normal(0, 1, X.shape))*math.sqrt(2*D_tracer*dt)
    x1 = x + D_bath/T*f1*dt + g0
    X1 = X + D_tracer/T*F1*dt + G0
    f2 = f(x1)
    bond_f2 = bond_k * (X1.reshape((1,-1)) - x1) / x.shape[0]
    f2 += bond_f2
    F2 = -bond_f2.sum(axis=0)

    return (x + D_bath/(2*T)*(f1+f2)*dt + g0, X + D_tracer/(2*T)*(F1+F2)*dt + G0)


In [ ]:
def run_w_tracer(x0, X0, k, f, dt=0.1, Nsteps=10000):

    data = [x0]
    DATA = [X0]
    x = x0.copy()
    X = X0.copy()
    for t in range(Nsteps):
        x, X = srk_w_tracer(x, X, dt, k, f)
        data.append(x)
        DATA.append(X)

    return np.array(data), np.array(DATA)

In [ ]:
x0 = np.zeros((32, 2))
X0 = np.zeros((2,))

x_data, X_data = run_w_tracer(x0, X0, 0.1, f_zero)

In [ ]:
[plt.plot(x_data[:,i,0], x_data[:,i,1]) for i in range(x_data.shape[1])];
plt.plot(X_data[:,0], X_data[:,1], 'k', lw=3);

In [ ]:
x0 = np.zeros((32, 2))
X0 = np.zeros((2,))

x_data, X_data = run_w_tracer(x0, X0, 1., f_hat, Nsteps=5000000, dt=0.02)

In [ ]:
[plt.plot(x_data[-2000:,i,0], x_data[-2000:,i,1]) for i in range(x_data.shape[1])];
plt.plot(X_data[:,0], X_data[:, 1], 'k', lw=3, alpha=0.6)

In [ ]:
plt.plot(X_data, 'k', lw=3, alpha=0.6)

In [ ]:
X_data.shape

In [ ]:
plt.hist2d(x_data[...,0].reshape((-1,)), x_data[...,1].reshape((-1,)), bins=32);
plt.figure()
plt.hist2d(X_data[:,0], X_data[:,1]);

In [ ]:

xx = np.empty( (100, 2))
xx[:,0] = np.arange(100)*0.05
xx[:,1] = 0

f = create_f_hat(1, 0.1)
ff = f(xx)
plt.plot(xx[:,0], ff[:,0])
#plt.plot(xx[:,0], ff[:,1])

f = create_f_hat(1, 1.)
ff = f(xx)
plt.plot(xx[:,0], ff[:,0])
#plt.plot(xx[:,0], ff[:,1])

#f = create_f_hat_rotation(1, 1.2, 1)
#ff = f(xx)
#plt.plot(xx[:,0], ff[:,0])
#plt.plot(xx[:,0], ff[:,1])

plt.ylim(-2, 2);

In [ ]:
import brownian_wrapper

In [ ]:
x0 = np.zeros((32,2))
X0 = np.zeros(2)

sims = []
for i in range(5):
    sims.append(brownian_wrapper.srk_with_tracer(x0, X0, 1, 0.001, 0.01, 400, 1000000, 1., 0.1, 0.5))

In [ ]:
plt.plot(sims[0][1]);

In [ ]:
plt.plot(x.mean(axis=1));

In [ ]:
plt.hist2d(x[...,0].reshape((-1,)), x[...,1].reshape((-1,)), bins=32);

In [ ]:
x_sq = x**2
x_sqrt = np.sqrt(np.sum(x_sq.reshape((-1,2)), axis=1))
plt.hist(x_sqrt, bins=32, normed=True, weights=1/x_sqrt);

In [ ]:
x.size

In [ ]:
[plt.plot(x[:,i,0], x[:,i,1]) for i in range(x.shape[1])];
plt.plot(X[:,0], X[:, 1], 'k', lw=3, alpha=0.6);

In [ ]:
plt.hist2d(X[:,0], X[:,1], bins=32);

In [ ]:
import brownian_wrapper

In [ ]:
x0 = np.zeros((1,2))
X0 = np.zeros(2)

x, X, force, force_count = brownian_wrapper.srk_with_tracer(x0, X0, 1, 0.005, 0.01, 100, 2500000, 1., 0.1, 0.5, 3., 0)

In [ ]:
plt.subplot(221)
plt.hist2d(X[:,0], X[:,1])
radius = np.sqrt(np.sum(X**2, axis=1))
plt.subplot(222)
plt.hist(radius, bins=32, normed=True, weights=1/radius);
plt.subplot(223)
plt.plot(np.arange(force.shape[0])*1./force.shape[0], force/force_count)
plt.subplot(224)
plt.hist2d(x[:,0,0], x[:,0,1], bins=32, normed=True);